# Decision Tree
In this notebook all features and outputs only take two values for simplicity. Full implementation is not here.
Also, here, I have implemented classification Decision Tree and not applied to regression problems.

In [1]:
import numpy as np

## Measuring impurity (Entropy)
Let's denote $p_1$ to be the fraction of class1 out of 2 classes - class0 and class1.<br>
So then $p_0 = 1 - p_1$.<br>
$
\begin{equation}
    H(p_1) = -p_1 log_2(p1) - (1 - p_1) log_2(1 - p_1)
\end{equation}
$
<br>
We can also write it as: <br>
$
    H(p_1) -p_1 log_2(p_1) - (p_0) log_2(p_0)
$

In [2]:
def compute_entropy(y):
    # Takes classes of the result and calculates purity; numpy narray and only takes 0 and 1
    m = y.shape[0]
    p_1 = y[y == 1].size
    p_1 = p_1 / m if m != 0 else 0
    if p_1 == 0 or p_1 == 1: return 0
    else: entropy = -p_1 * np.log2(p_1) - (1 - p_1) * np.log2(1 - p_1)
    
    return entropy

## Splitting
Each node contains some fraction of the Data (rows) and they themselves split untill criteria is met.
We split based on a feature. If some data has the feature so split to left; otherwise we split to the right sub-branch.

In [3]:
def split(X, node_indices, on_feature):
    # Matrix containing m, n rows and columns respectively.
    # list containing indices in the current node
    # int, as the index of columns in X
    left = []
    right = []
    
    for i in node_indices:
        if X[i][on_feature] == 1:
            left.append(i)
        else: right.append(i)
    
    return left, right

## Information Gain
Information gain is actually the "Reduction of average weighted entropy compared to the parent or root of the tree". We calculate weighted entropy as the fraction of data in the left compared to the parent times the entropy plus the fraction of data in the right compared to the parent times entropy. Information gain, then will be: <br>
$
    \text{Information Gain} = H(p_1^{\text{root}}) - (w_{\text{left}} H(p_1^{\text{left}}) + w_{\text{right}} H(p_1^{\text{right}}))
$

In [4]:
def compute_information_gain(X, y, node_indices, on_feature):
    # ndarray matrix consist of m and n rows and columns respectively
    # array consisting of 1 row and m columns
    # list of current indicies inside the node
    # in, as the index of columns in X to perform split
    
    # split
    left_indices, right_indices = split(X, node_indices, on_feature) 
    X_root, y_root = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
    # calculate weights
    w_left = X_left.shape[0] / X_root.shape[0]
    w_right = X_right.shape[0] / X_root.shape[0]
    
    # calculate entropies
    e_left = compute_entropy(y_left)
    e_right = compute_entropy(y_right)
    e_root = compute_entropy(y_root)
    
    # compute information gain
    ig = e_root - (w_left * e_left + w_right * e_right)
    
    return ig

## Choosing the best feature
The best feature is the one that has the most information gain (i.e. reduces entropy or impurity compared to the parent or the root). If a node is completely pure, we do not split anymore.

In [5]:
def choose_best_feature(X, y, node_indices):
    # ndarray matrix consist of m and n rows and columns respectively
    # array consisting of 1 row and m columns
    # list of current indicies inside the node
    
    m, n = X.shape
    
    # see whether current node is pure
    if compute_entropy(y[node_indices]) == 0: return -1
    
    # find the best feature to maximize the information_gain
    max_info = -1
    best_feature = -1
    for i in range(n):
        info = compute_information_gain(X, y, node_indices, i)
        if info > max_info:
            max_info = info
            best_feature = i
    
    return best_feature

## Build the tree
Recursively split each node untill each of them meet the criteria and then stop splitting.

In [6]:
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    if current_depth == max_depth: return # Base case: Criteria is met
    
    # choose the best feature to split base on that
    b_f = choose_best_feature(X, y, node_indices)
    if b_f == -1: return # Base case: Criteria is met (Pure node)
    
    # building the tree
    tree.append([current_depth, branch_name, b_f, node_indices])
    
    # split data
    left_indices, right_indices = split(X, node_indices, b_f)
    
    # recursive calls on left and right branches
    build_tree_recursive(X, y, left_indices, 'Left', max_depth, current_depth + 1)
    build_tree_recursive(X, y, right_indices, 'Right', max_depth, current_depth + 1)

## Example

In [7]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [8]:
build_tree_recursive(X_train, y_train, list(range(X_train.shape[0])), 'root', 4, 0)

In [9]:
tree

[[0, 'root', 2, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
 [1, 'Left', 0, [0, 1, 4, 5, 7]],
 [1, 'Right', 1, [2, 3, 6, 8, 9]]]

End!